# EfficientDet-D1 과일 품질 등급 분류 - 모델 평가
## 목표: 과일 종류와 등급을 분류하고 YOLOv5s와 성능 비교

**Classes:**
- 0: 사과_특, 1: 사과_상, 2: 사과_중
- 3: 배_특, 4: 배_상, 5: 배_중
- 6: 감_특, 7: 감_상, 8: 감_중

In [1]:
## 1. 환경 설정 및 라이브러리 Import
import os
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import time
from tqdm import tqdm
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    f1_score, confusion_matrix, classification_report,
    precision_recall_fscore_support
)
import pandas as pd
import cv2

# EfficientDet 관련
from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
from effdet.efficientdet import HeadNet

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device('cpu')
print(f"Using device: {device}")

PyTorch version: 2.9.0
CUDA available: False
Using device: cpu


In [2]:
## 2. 경로 및 클래스 설정
# 경로 설정
BASE_DIR = Path('../../data')
DATASET_DIR = BASE_DIR / 'row'
IMAGES_DIR = DATASET_DIR / 'images'
LABELS_DIR = DATASET_DIR / 'json_labels'

# 모델 경로
MODEL_DIR = Path('../../models')
MODEL_CATE1_PATH = MODEL_DIR / 'efficientdet_d1_cate1_best.pth'
MODEL_CATE3_PATH = MODEL_DIR / 'efficientdet_d1_cate3_best.pth'
MODEL_COMBINED_PATH = MODEL_DIR / 'efficientdet_d1_combined_best.pth'

# 클래스 정의
CLASSES_CATE1 = ['사과', '배', '감']
CLASSES_CATE3 = ['특', '상', '중']
CLASSES_COMBINED = [
    '사과_특', '사과_상', '사과_중',
    '배_특', '배_상', '배_중',
    '감_특', '감_상', '감_중'
]

print(f"Dataset directory: {DATASET_DIR}")
print(f"Images directory: {IMAGES_DIR}")
print(f"Labels directory: {LABELS_DIR}")

Dataset directory: ../../data/row
Images directory: ../../data/row/images
Labels directory: ../../data/row/json_labels


In [9]:
## 3. 테스트 데이터 로드
def load_test_data(test_split_file='test_split.txt'):
    """테스트 데이터 로드"""
    test_files = []
    
    if os.path.exists(test_split_file):
        with open(test_split_file, 'r') as f:
            test_files = [line.strip() for line in f.readlines()]
    else:
        all_json_files = sorted(LABELS_DIR.glob('*.json'))
        np.random.seed(42)
        test_indices = np.random.choice(len(all_json_files), size=int(len(all_json_files)*0.1), replace=False)
        test_files = [all_json_files[i].stem for i in test_indices]
    
    test_data = []
    
    for file_name in tqdm(test_files, desc="Loading test data"):
        json_path = LABELS_DIR / f"{file_name}.json"
        img_path = IMAGES_DIR / f"{file_name}.png"
        
        if not img_path.exists():
            img_path = IMAGES_DIR / f"{file_name}.jpg"
        
        if not json_path.exists() or not img_path.exists():
            continue
        
        with open(json_path, 'r', encoding='utf-8') as f:
            label_data = json.load(f)
        
        test_data.append({
            'file_name': file_name,
            'img_path': str(img_path),
            'cate1': label_data['cate1'],
            'cate3': label_data['cate3'],
            'combined': f"{label_data['cate1']}_{label_data['cate3']}",
            'bbox': label_data['bndbox']
        })
    
    print(f"\n총 테스트 데이터: {len(test_data)}개")
    return test_data

# 테스트 데이터 로드
test_data = load_test_data()

# 데이터 분포 확인
cate1_dist = pd.Series([d['cate1'] for d in test_data]).value_counts()
cate3_dist = pd.Series([d['cate3'] for d in test_data]).value_counts()
combined_dist = pd.Series([d['combined'] for d in test_data]).value_counts()

print("\n[과일 종류 분포]")
print(cate1_dist)
print("\n[등급 분포]")
print(cate3_dist)
print("\n[통합 분포]")
print(combined_dist)

Loading test data: 100%|██████████| 7/7 [00:00<00:00, 1880.73it/s]


총 테스트 데이터: 7개

[과일 종류 분포]
사과    5
감     2
Name: count, dtype: int64

[등급 분포]
특     3
상     3
보통    1
Name: count, dtype: int64

[통합 분포]
사과_특     2
사과_상     2
사과_보통    1
감_특      1
감_상      1
Name: count, dtype: int64


In [10]:
## 4. 모델 로드
def load_efficientdet_model(model_path, num_classes):
    """EfficientDet 모델 로드"""
    config = get_efficientdet_config('tf_efficientdet_d1')
    config.num_classes = num_classes
    config.image_size = 512
    
    net = EfficientDet(config, pretrained_backbone=False)
    net.class_net = HeadNet(config, num_outputs=config.num_classes)
    
    checkpoint = torch.load(model_path, map_location=device)
    net.load_state_dict(checkpoint['model_state_dict'])
    
    net.eval()
    net.to(device)
    
    print(f"모델 로드 완료: {model_path}")
    print(f"클래스 수: {num_classes}")
    
    return net, config

# 3가지 모델 로드
print("\n=== 모델 로드 시작 ===")
model_cate1, config_cate1 = load_efficientdet_model(MODEL_CATE1_PATH, len(CLASSES_CATE1))
model_cate3, config_cate3 = load_efficientdet_model(MODEL_CATE3_PATH, len(CLASSES_CATE3))
model_combined, config_combined = load_efficientdet_model(MODEL_COMBINED_PATH, len(CLASSES_COMBINED))
print("=== 모델 로드 완료 ===")


=== 모델 로드 시작 ===


TypeError: 'int' object is not subscriptable

In [ ]:
## 5. 추론 및 평가 함수
def preprocess_image(img_path, image_size=512):
    """이미지 전처리"""
    img = Image.open(img_path).convert('RGB')
    img = img.resize((image_size, image_size))
    img_array = np.array(img).astype(np.float32) / 255.0
    img_tensor = torch.from_numpy(img_array).permute(2, 0, 1).unsqueeze(0)
    return img_tensor.to(device), img

def predict_single_image(model, img_path, conf_threshold=0.5):
    """단일 이미지 추론"""
    img_tensor, original_img = preprocess_image(img_path)
    
    with torch.no_grad():
        output = model(img_tensor)
    
    boxes = output[0]['boxes'].cpu().numpy() if len(output[0]['boxes']) > 0 else np.array([])
    scores = output[0]['scores'].cpu().numpy() if len(output[0]['scores']) > 0 else np.array([])
    labels = output[0]['labels'].cpu().numpy() if len(output[0]['labels']) > 0 else np.array([])
    
    if len(scores) > 0:
        mask = scores >= conf_threshold
        boxes = boxes[mask]
        scores = scores[mask]
        labels = labels[mask]
    
    if len(scores) > 0:
        best_idx = np.argmax(scores)
        return int(labels[best_idx]), float(scores[best_idx]), boxes[best_idx]
    else:
        return -1, 0.0, None

def evaluate_model(model, test_data, class_names, task_name):
    """모델 평가 수행"""
    print(f"\n{'='*60}")
    print(f"평가 시작: {task_name}")
    print(f"{'='*60}")
    
    y_true = []
    y_pred = []
    confidences = []
    inference_times = []
    
    for data in tqdm(test_data, desc=f"{task_name} 추론 중"):
        if task_name == "과일 종류 분류 (cate1)":
            true_label = data['cate1']
        elif task_name == "등급 분류 (cate3)":
            true_label = data['cate3']
        else:
            true_label = data['combined']
        
        true_idx = class_names.index(true_label)
        
        start_time = time.time()
        pred_idx, conf, bbox = predict_single_image(model, data['img_path'])
        inference_time = time.time() - start_time
        
        y_true.append(true_idx)
        y_pred.append(pred_idx if pred_idx != -1 else 0)
        confidences.append(conf)
        inference_times.append(inference_time)
    
    # 메트릭 계산
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    avg_inference_time = np.mean(inference_times)
    fps = 1.0 / avg_inference_time if avg_inference_time > 0 else 0
    avg_confidence = np.mean(confidences)
    
    cm = confusion_matrix(y_true, y_pred)
    
    print(f"\n[{task_name} 평가 결과]")
    print(f"정확도(Accuracy): {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"정밀도(Precision): {precision:.4f}")
    print(f"재현율(Recall): {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"평균 Confidence: {avg_confidence:.4f}")
    print(f"평균 추론 시간: {avg_inference_time:.4f}초")
    print(f"FPS: {fps:.2f}")
    
    print("\n[상세 분류 리포트]")
    print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))
    
    results = {
        'task_name': task_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'avg_confidence': avg_confidence,
        'avg_inference_time': avg_inference_time,
        'fps': fps,
        'confusion_matrix': cm,
        'y_true': y_true,
        'y_pred': y_pred,
        'class_names': class_names
    }
    
    return results

In [ ]:
## 6. 모델 평가 실행
# 3가지 모델 평가
results_cate1 = evaluate_model(model_cate1, test_data, CLASSES_CATE1, "과일 종류 분류 (cate1)")
results_cate3 = evaluate_model(model_cate3, test_data, CLASSES_CATE3, "등급 분류 (cate3)")
results_combined = evaluate_model(model_combined, test_data, CLASSES_COMBINED, "통합 분류 (Combined)")

all_results = {
    'cate1': results_cate1,
    'cate3': results_cate3,
    'combined': results_combined
}

In [ ]:
## 7. 혼동 행렬 시각화
def plot_confusion_matrix(cm, class_names, title):
    """혼동 행렬 시각화"""
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': '예측 수'})
    plt.title(f'혼동 행렬 - {title}', fontsize=16, pad=20)
    plt.ylabel('실제 클래스', fontsize=12)
    plt.xlabel('예측 클래스', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_{title.replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_confusion_matrix(results_cate1['confusion_matrix'], CLASSES_CATE1, "과일 종류 분류")
plot_confusion_matrix(results_cate3['confusion_matrix'], CLASSES_CATE3, "등급 분류")
plot_confusion_matrix(results_combined['confusion_matrix'], CLASSES_COMBINED, "통합 분류")

In [ ]:
## 8. 성능 메트릭 비교
from math import pi

metrics_df = pd.DataFrame({
    '모델': ['과일 종류 분류', '등급 분류', '통합 분류'],
    '정확도': [results_cate1['accuracy'], results_cate3['accuracy'], results_combined['accuracy']],
    '정밀도': [results_cate1['precision'], results_cate3['precision'], results_combined['precision']],
    '재현율': [results_cate1['recall'], results_cate3['recall'], results_combined['recall']],
    'F1-Score': [results_cate1['f1_score'], results_cate3['f1_score'], results_combined['f1_score']],
    'FPS': [results_cate1['fps'], results_cate3['fps'], results_combined['fps']]
})

print("\n=== EfficientDet-D1 성능 비교 ===")
print(metrics_df.to_string(index=False))

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 정확도 비교
axes[0, 0].bar(metrics_df['모델'], metrics_df['정확도'], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0, 0].set_title('정확도 비교', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('정확도', fontsize=12)
axes[0, 0].set_ylim([0, 1])
for i, v in enumerate(metrics_df['정확도']):
    axes[0, 0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10)

# 정밀도/재현율/F1 비교
x = np.arange(len(metrics_df['모델']))
width = 0.25
axes[0, 1].bar(x - width, metrics_df['정밀도'], width, label='정밀도', color='#3498db')
axes[0, 1].bar(x, metrics_df['재현율'], width, label='재현율', color='#e74c3c')
axes[0, 1].bar(x + width, metrics_df['F1-Score'], width, label='F1-Score', color='#2ecc71')
axes[0, 1].set_title('정밀도/재현율/F1-Score 비교', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(metrics_df['모델'], rotation=15, ha='right')
axes[0, 1].set_ylabel('점수', fontsize=12)
axes[0, 1].set_ylim([0, 1])
axes[0, 1].legend()

# FPS 비교
axes[1, 0].bar(metrics_df['모델'], metrics_df['FPS'], color=['#9b59b6', '#f39c12', '#1abc9c'])
axes[1, 0].set_title('추론 속도(FPS) 비교', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('FPS', fontsize=12)
for i, v in enumerate(metrics_df['FPS']):
    axes[1, 0].text(i, v + 0.05, f'{v:.2f}', ha='center', fontsize=10)

# 레이더 차트
categories = ['정확도', '정밀도', '재현율', 'F1-Score']
N = len(categories)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

ax = plt.subplot(2, 2, 4, projection='polar')
for idx, model_name in enumerate(metrics_df['모델']):
    values = metrics_df.iloc[idx][['정확도', '정밀도', '재현율', 'F1-Score']].values.tolist()
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=model_name)
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1)
ax.set_title('종합 성능 비교 (레이더 차트)', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.savefig('efficientdet_performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
## 9. 예측 결과 시각화
def visualize_predictions(model, test_data, class_names, task_name, num_samples=9):
    """예측 결과 시각화"""
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    axes = axes.ravel()
    
    sample_indices = np.random.choice(len(test_data), min(num_samples, len(test_data)), replace=False)
    
    for idx, sample_idx in enumerate(sample_indices):
        data = test_data[sample_idx]
        
        if task_name == "과일 종류 분류":
            true_label = data['cate1']
        elif task_name == "등급 분류":
            true_label = data['cate3']
        else:
            true_label = data['combined']
        
        pred_idx, conf, bbox = predict_single_image(model, data['img_path'])
        pred_label = class_names[pred_idx] if pred_idx != -1 else "예측 실패"
        
        img = Image.open(data['img_path'])
        
        axes[idx].imshow(img)
        axes[idx].axis('off')
        
        is_correct = (pred_label == true_label)
        color = 'green' if is_correct else 'red'
        
        title = f"실제: {true_label}\\n예측: {pred_label}\\nConf: {conf:.3f}"
        axes[idx].set_title(title, fontsize=10, color=color, fontweight='bold')
    
    plt.suptitle(f'{task_name} - 예측 결과 샘플', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'predictions_{task_name.replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
    plt.show()

visualize_predictions(model_cate1, test_data, CLASSES_CATE1, "과일 종류 분류")
visualize_predictions(model_cate3, test_data, CLASSES_CATE3, "등급 분류")
visualize_predictions(model_combined, test_data, CLASSES_COMBINED, "통합 분류")

In [ ]:
## 10. YOLOv5s와 성능 비교
# YOLOv5 결과 (수동 입력 또는 파일 로드)
YOLO_RESULTS_PATH = Path('./yolo_results.json')

yolo_results = {
    'model_name': 'YOLOv5s',
    'accuracy': 0.0,
    'precision': 0.0,
    'recall': 0.0,
    'f1_score': 0.0,
    'fps': 0.0
}

if YOLO_RESULTS_PATH.exists():
    with open(YOLO_RESULTS_PATH, 'r') as f:
        yolo_results = json.load(f)
    print("YOLOv5 결과 로드 완료")
else:
    print("YOLOv5 결과 파일이 없습니다. 수동으로 입력하세요:")
    try:
        yolo_results['accuracy'] = float(input("정확도(Accuracy): ") or "0.0")
        yolo_results['precision'] = float(input("정밀도(Precision): ") or "0.0")
        yolo_results['recall'] = float(input("재현율(Recall): ") or "0.0")
        yolo_results['f1_score'] = float(input("F1-Score: ") or "0.0")
        yolo_results['fps'] = float(input("FPS: ") or "0.0")
    except:
        print("입력을 건너뛰었습니다.")

print("\n=== YOLOv5s 성능 ===")
print(f"정확도: {yolo_results['accuracy']:.4f}")
print(f"정밀도: {yolo_results['precision']:.4f}")
print(f"재현율: {yolo_results['recall']:.4f}")
print(f"F1-Score: {yolo_results['f1_score']:.4f}")
print(f"FPS: {yolo_results['fps']:.2f}")

In [ ]:
## 11. EfficientDet vs YOLOv5s 비교 시각화
comparison_df = pd.DataFrame({
    '모델': ['EfficientDet-D1', 'YOLOv5s'],
    '정확도': [results_combined['accuracy'], yolo_results['accuracy']],
    '정밀도': [results_combined['precision'], yolo_results['precision']],
    '재현율': [results_combined['recall'], yolo_results['recall']],
    'F1-Score': [results_combined['f1_score'], yolo_results['f1_score']],
    'FPS': [results_combined['fps'], yolo_results['fps']]
})

print("\n=== EfficientDet-D1 vs YOLOv5s 비교 ===")
print(comparison_df.to_string(index=False))

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
metrics = ['정확도', '정밀도', '재현율', 'F1-Score', 'FPS']
colors = ['#3498db', '#e74c3c']

for idx, metric in enumerate(metrics):
    row = idx // 3
    col = idx % 3
    
    axes[row, col].bar(comparison_df['모델'], comparison_df[metric], color=colors)
    axes[row, col].set_title(f'{metric} 비교', fontsize=14, fontweight='bold')
    axes[row, col].set_ylabel(metric, fontsize=12)
    
    for i, v in enumerate(comparison_df[metric]):
        if metric == 'FPS':
            axes[row, col].text(i, v + max(comparison_df[metric])*0.02, f'{v:.2f}', 
                              ha='center', fontsize=11, fontweight='bold')
        else:
            axes[row, col].text(i, v + 0.02, f'{v:.3f}', 
                              ha='center', fontsize=11, fontweight='bold')
    
    if metric != 'FPS':
        axes[row, col].set_ylim([0, 1.1])

# 레이더 차트
categories = ['정확도', '정밀도', '재현율', 'F1-Score']
N = len(categories)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

ax = plt.subplot(2, 3, 6, projection='polar')

for idx, model_name in enumerate(comparison_df['모델']):
    values = comparison_df.iloc[idx][['정확도', '정밀도', '재현율', 'F1-Score']].values.tolist()
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=model_name, color=colors[idx])
    ax.fill(angles, values, alpha=0.25, color=colors[idx])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1)
ax.set_title('종합 성능 비교', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.suptitle('EfficientDet-D1 vs YOLOv5s 성능 비교', fontsize=18, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('efficientdet_vs_yolo_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
## 12. 결론 및 분석
print("\n" + "="*80)
print("최종 분석 및 결론")
print("="*80)

accuracy_diff = results_combined['accuracy'] - yolo_results['accuracy']
fps_diff = results_combined['fps'] - yolo_results['fps']

print("\n[1. 정확도 비교]")
print(f"  - EfficientDet-D1: {results_combined['accuracy']:.4f} ({results_combined['accuracy']*100:.2f}%)")
print(f"  - YOLOv5s: {yolo_results['accuracy']:.4f} ({yolo_results['accuracy']*100:.2f}%)")
print(f"  - 차이: {accuracy_diff:+.4f} ({accuracy_diff*100:+.2f}%p)")

print("\n[2. 추론 속도 비교]")
print(f"  - EfficientDet-D1: {results_combined['fps']:.2f} FPS")
print(f"  - YOLOv5s: {yolo_results['fps']:.2f} FPS")
print(f"  - 차이: {fps_diff:+.2f} FPS")

print("\n[3. 과일 품질 분류 태스크에 대한 결론]")
print("  ■ 데이터셋 특성:")
print("    - 총 70개 샘플 (사과: 42.9%, 배: 14.3%, 감: 42.9%)")
print("    - 9개 클래스 (3가지 과일 × 3가지 등급)")
print("\\n  ■ 향후 개선 방향:")
print("    1. 데이터 증강으로 학습 데이터 확보")
print("    2. 배 데이터 추가 수집 (현재 14.3%)")
print("    3. 하이퍼파라미터 튜닝")
print("    4. 앙상블 기법 적용")

# 클래스별 성능 분석
precision_per_class, recall_per_class, f1_per_class, support_per_class = precision_recall_fscore_support(
    results_combined['y_true'], 
    results_combined['y_pred'],
    labels=range(len(CLASSES_COMBINED)),
    zero_division=0
)

class_performance_df = pd.DataFrame({
    '클래스': CLASSES_COMBINED,
    '정밀도': precision_per_class,
    '재현율': recall_per_class,
    'F1-Score': f1_per_class,
    '샘플 수': support_per_class
})

print("\n=== 클래스별 성능 분석 ===")
print(class_performance_df.to_string(index=False))

# 클래스별 성능 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].barh(class_performance_df['클래스'], class_performance_df['정밀도'], color='skyblue')
axes[0].set_xlabel('정밀도', fontsize=12)
axes[0].set_title('클래스별 정밀도', fontsize=14, fontweight='bold')
axes[0].set_xlim([0, 1])

axes[1].barh(class_performance_df['클래스'], class_performance_df['재현율'], color='lightcoral')
axes[1].set_xlabel('재현율', fontsize=12)
axes[1].set_title('클래스별 재현율', fontsize=14, fontweight='bold')
axes[1].set_xlim([0, 1])

axes[2].barh(class_performance_df['클래스'], class_performance_df['F1-Score'], color='lightgreen')
axes[2].set_xlabel('F1-Score', fontsize=12)
axes[2].set_title('클래스별 F1-Score', fontsize=14, fontweight='bold')
axes[2].set_xlim([0, 1])

plt.tight_layout()
plt.savefig('class_wise_performance.png', dpi=300, bbox_inches='tight')
plt.show()

# 최고/최저 성능 클래스
best_class_idx = class_performance_df['F1-Score'].idxmax()
worst_class_idx = class_performance_df['F1-Score'].idxmin()

print(f"\n[최고 성능 클래스]")
print(f"  {class_performance_df.iloc[best_class_idx]['클래스']}: F1-Score = {class_performance_df.iloc[best_class_idx]['F1-Score']:.3f}")

print(f"\n[최저 성능 클래스]")
print(f"  {class_performance_df.iloc[worst_class_idx]['클래스']}: F1-Score = {class_performance_df.iloc[worst_class_idx]['F1-Score']:.3f}")
print(f"  → 개선 필요: 해당 클래스의 데이터 증강 또는 추가 수집 권장")

In [ ]:
## 13. 결과 저장
# 최종 결과를 JSON으로 저장
final_results = {
    'efficientdet_d1': {
        'cate1': {
            'accuracy': float(results_cate1['accuracy']),
            'precision': float(results_cate1['precision']),
            'recall': float(results_cate1['recall']),
            'f1_score': float(results_cate1['f1_score']),
            'fps': float(results_cate1['fps'])
        },
        'cate3': {
            'accuracy': float(results_cate3['accuracy']),
            'precision': float(results_cate3['precision']),
            'recall': float(results_cate3['recall']),
            'f1_score': float(results_cate3['f1_score']),
            'fps': float(results_cate3['fps'])
        },
        'combined': {
            'accuracy': float(results_combined['accuracy']),
            'precision': float(results_combined['precision']),
            'recall': float(results_combined['recall']),
            'f1_score': float(results_combined['f1_score']),
            'fps': float(results_combined['fps'])
        }
    },
    'yolov5s': yolo_results
}

# JSON 파일로 저장
output_path = 'final_evaluation_results.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(final_results, f, ensure_ascii=False, indent=2)

print(f"\n평가 결과가 저장되었습니다: {output_path}")

# Excel로도 저장
try:
    with pd.ExcelWriter('final_evaluation_results.xlsx', engine='openpyxl') as writer:
        metrics_df.to_excel(writer, sheet_name='EfficientDet_비교', index=False)
        comparison_df.to_excel(writer, sheet_name='EfficientDet_vs_YOLO', index=False)
        class_performance_df.to_excel(writer, sheet_name='클래스별_성능', index=False)
    print("Excel 파일도 저장되었습니다: final_evaluation_results.xlsx")
except:
    print("Excel 저장 실패 (openpyxl 라이브러리가 필요합니다)")

print("\n=== 평가 완료 ===")

## 14. 프로젝트 요약

### 프로젝트 목표
소비자의 의사결정을 돕기 위한 과일 품질 등급 분류 시스템 개발

### 사용 모델
- **EfficientDet-D1**: Compound Scaling과 BiFPN을 활용한 효율적인 객체 탐지
- **YOLOv5s**: 빠른 추론 속도를 가진 실시간 객체 탐지 모델

### 데이터셋
- 총 70개 샘플 (사과 42.9%, 배 14.3%, 감 42.9%)
- 3가지 과일 종류 × 3가지 등급 = 9개 클래스

### 평가 결과
위 셀들의 실행 결과 참조

### 결론
- **정확도 우선**: EfficientDet-D1 추천 (높은 분류 정확도)
- **속도 우선**: YOLOv5s 추천 (빠른 추론 속도)
- **실제 적용**: 동네 마트 환경과 요구사항에 따라 선택

### 향후 개선 방향
1. 데이터 증강을 통한 학습 데이터 확보
2. 배 데이터 추가 수집 (현재 불균형)
3. 하이퍼파라미터 최적화
4. 앙상블 기법으로 두 모델의 장점 결합
5. 실시간 웹/모바일 애플리케이션 개발